In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

24/08/08 14:09:48 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/08 14:09:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 14:09:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/08 14:09:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/08 14:09:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/08 14:09:50 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/08 14:09:50 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [3]:
sentenceData = spark.createDataFrame([
    (0.0, 'Hi I heard about Spark'), 
    (0.0, 'I wish java could use case classes'),
    (1.0, 'Logistic regression models are neat')],
    ['label', 'sentence']) 

In [5]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol = 'sentence', outputCol = 'words')

In [7]:
words_data = tokenizer.transform(sentenceData)

In [8]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [9]:
words_data.show(truncate= False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [10]:
hashing_tf = HashingTF(inputCol = 'words', outputCol = 'rawFeatures')

In [11]:
featurized_data = hashing_tf.transform(words_data)

In [12]:
idf = IDF(inputCol = 'rawFeatures', outputCol = 'features')

In [14]:
idf_model = idf.fit(featurized_data)

In [15]:
rescaled_data = idf_model.transform(featurized_data)

In [16]:
rescaled_data.select('label', 'features').show()

24/08/08 14:15:21 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/08/08 14:15:21 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/08/08 14:15:21 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[18700,19...|
|  0.0|(262144,[19036,20...|
|  1.0|(262144,[46243,58...|
+-----+--------------------+



In [17]:
rescaled_data.select('label', 'features').show(truncate= False)

24/08/08 14:15:41 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/08/08 14:15:42 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/08/08 14:15:42 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

In [18]:
from pyspark.ml.feature import CountVectorizer

In [19]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [20]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [22]:
cv = CountVectorizer(inputCol = 'words', outputCol = 'features', vocabSize = 3, minDF = 2.0)

In [23]:
model = cv.fit(df)

In [24]:
result = model.transform(df)

In [25]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

